In [1]:
from cluster_model_state import FedStateClusterConfig, FedStateCluster
from data_utils import SimpleDataset, split_noniid

import torch
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
config = FedStateClusterConfig(
    n_clients=2,
    n_clusters=2,
    n_classes=10,
    partition_column='label',
    client_lr=1e-3,
    dataset='rotmnist',
    method='dirichlet',
    dirichlet_alpha=0.5,
    m=1,
    global_rounds=20,
    client_epochs=3,
    client_bs=64,
    global_bs=128,
    train_test_split = 0.8,
    total_train_samples = 100000,
    local_eval_every = 1,
    cluster_every = 3,
    start_recluster = 10
)

torch.manual_seed(config.torch_seed)
np.random.seed(config.np_seed)


In [3]:
train_data = datasets.MNIST(root="../../data/mnist", download=False, train=True)
test_data = datasets.MNIST(root="../../data/mnist",  download=False, train=False)

mapp = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U1')

# mapp = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C',
#     'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P',
#     'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c',
#     'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
#     'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'], dtype='<U1')

In [4]:
idcs = np.random.permutation(len(train_data))    

all_cleint_idcs = idcs[:config.total_train_samples]

per_client_samples = len(all_cleint_idcs) // config.n_clients

train_labels = train_data.train_labels.numpy()

client_idcs = split_noniid(all_cleint_idcs, train_labels, alpha=config.dirichlet_alpha, n_clients=config.n_clients)


C:\Users\kkire\anaconda3\envs\dl\Lib\site-packages\torchvision\datasets\mnist.py:66: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [19]:
client_data = [Subset(train_data, idcs) for idcs in client_idcs]    

client_train_partitions, client_test_partitions = {}, {}

for i in range(config.n_clients):        
    n_train= int(config.train_test_split*len(client_data[i]))
    n_eval =  len(client_data[i]) - n_train
    client_i_train, client_i_test = torch.utils.data.random_split(client_data[i], [n_train, n_eval])
    
    # to create rotated MNIST
    if i < (0.5*config.n_clients):
        client_train_partitions[i] = SimpleDataset(client_i_train, transforms.Compose([transforms.RandomRotation((180,180)),
                                                    transforms.ToTensor()]))
    else:        
        client_train_partitions[i] = SimpleDataset(client_i_train, transforms.Compose([transforms.ToTensor()]))
    
    client_test_partitions[i] = SimpleDataset(client_i_test, transforms.Compose([transforms.ToTensor()]))        


In [20]:
from IPython.display import display
from PIL import Image

for sample in client_train_partitions[0]:
    img, label = sample
    display(Image.fromarray((img.squeeze()*255).numpy().astype(np.uint8)))
    print(label)
    break

9


In [21]:
global_test_data = SimpleDataset(Subset(test_data, np.arange(1000)), transforms.Compose([transforms.ToTensor()]))            

In [22]:
component = FedStateCluster(config, global_test_data, client_train_partitions, client_test_partitions)

Initializing with a single cluster for all clients.


In [23]:
component.models

[MNISTModel(
   (fc): Sequential(
     (0): Flatten(start_dim=1, end_dim=-1)
     (1): Linear(in_features=784, out_features=256, bias=True)
     (2): ReLU()
     (3): Linear(in_features=256, out_features=128, bias=True)
     (4): ReLU()
     (5): Linear(in_features=128, out_features=64, bias=True)
     (6): ReLU()
     (7): Linear(in_features=64, out_features=10, bias=True)
   )
 ),
 MNISTModel(
   (fc): Sequential(
     (0): Flatten(start_dim=1, end_dim=-1)
     (1): Linear(in_features=784, out_features=256, bias=True)
     (2): ReLU()
     (3): Linear(in_features=256, out_features=128, bias=True)
     (4): ReLU()
     (5): Linear(in_features=128, out_features=64, bias=True)
     (6): ReLU()
     (7): Linear(in_features=64, out_features=10, bias=True)
   )
 )]